In [15]:
import pandas as pd
import numpy as np
import geopandas as gpd

In [16]:
# import geopandas as gpd

# Read the IPC data
areas_gdf = gpd.read_file(r'C:\Users\WeilunShi\OneDrive - CGIAR\Desktop\Food Crisis and Insecurity\1.Initilization\intermediate\output.geojson')

In [17]:
areas_gdf.columns

Index(['id', 'estimated_population', 'period', 'from', 'to', 'color', 'anl_id',
       'title', 'overall_phase', 'country', 'year', 'condition',
       'phase3_worse_population', 'phase3_worse_percentage',
       'phase1_population', 'phase1_percent', 'phase1_color',
       'phase2_population', 'phase2_percent', 'phase2_color',
       'phase3_population', 'phase3_percent', 'phase3_color',
       'phase4_population', 'phase4_percent', 'phase4_color',
       'phase5_population', 'phase5_percent', 'phase5_color',
       'phase3_worse_percentage_manual', 'phase3_plus_phase4',
       'phase2_worse_percentage_manual', 'phase2_plus_phase3',
       'phase2_plus_phase3_plus_phase4', 'geometry'],
      dtype='object')

In [18]:
# Drop the columns that are not needed
areas_gdf = areas_gdf.drop(columns=['id','period','to','color','anl_id','year','condition','phase1_color','phase2_color','phase3_color','phase4_color','phase5_color'])

# Rename from to date
areas_gdf = areas_gdf.rename(columns={'from':'date'})

# Convert the date to datetime
areas_gdf['date'] = pd.to_datetime(areas_gdf['date'])

# convert geometry to coordinate reference WGS84
areas_gdf = areas_gdf.to_crs("EPSG:4326")

C:\Users\WeilunShi\AppData\Local\Temp\ipykernel_18500\1258726294.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  areas_gdf['date'] = pd.to_datetime(areas_gdf['date'])


In [19]:
areas_gdf

,estimated_population,date,title,overall_phase,country,phase3_worse_population,phase3_worse_percentage,phase1_population,phase1_percent,phase2_population,...,phase4_population,phase4_percent,phase5_population,phase5_percent,phase3_worse_percentage_manual,phase3_plus_phase4,phase2_worse_percentage_manual,phase2_plus_phase3,phase2_plus_phase3_plus_phase4,geometry
0,906391,2017-04-01,Dépressions de l`est,3,BI,271918,0.30,271917,0.30,362556,...,45320.0,0.05,0.0,0.0,0.30,0.30,0.70,0.65,0.70,"MULTIPOLYGON (((29.96857 -3.98761, 29.98180 -3..."
1,1772906,2017-04-01,Plateaux secs de l`est,3,BI,709163,0.40,354581,0.20,709162,...,177291.0,0.10,0.0,0.0,0.40,0.40,0.80,0.70,0.80,"MULTIPOLYGON (((30.27603 -2.97597, 30.27784 -2..."
2,238111,2017-04-01,Buragane,3,BI,47623,0.20,107150,0.45,83339,...,11906.0,0.05,0.0,0.0,0.20,0.20,0.55,0.50,0.55,"MULTIPOLYGON (((29.84228 -4.21076, 29.84333 -4..."
3,1171450,2017-04-01,Crête congo nil,2,BI,222575,0.19,410008,0.35,538867,...,58573.0,0.05,0.0,0.0,0.19,0.19,0.65,0.60,0.65,"MULTIPOLYGON (((29.36013 -2.95831, 29.36748 -2..."
4,651193,2017-04-01,Dépressions du nord,4,BI,293037,0.45,162798,0.25,195358,...,130239.0,0.20,0.0,0.0,0.45,0.45,0.75,0.55,0.75,"MULTIPOLYGON (((30.10073 -2.42500, 30.09402 -2..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9474,9604,2023-01-01,Zeylac Urban,2,SO,0,0.00,7680,0.80,1440,...,0.0,0.00,0.0,0.0,0.05,0.05,0.20,0.20,0.20,POINT (43.47261 11.35204)
9475,130615,2023-02-01,Ocotepeque,2,LAC,0,0.00,96655,0.74,19592,...,1306.0,0.01,0.0,0.0,0.11,0.11,0.26,0.25,0.26,"MULTIPOLYGON (((-89.02219 14.30899, -89.02218 ..."
9476,12705,2023-02-01,Cayaguanca,2,LAC,0,0.00,9529,0.75,2033,...,127.0,0.01,0.0,0.0,0.09,0.09,0.25,0.24,0.25,"MULTIPOLYGON (((-89.12431 14.28521, -89.12204 ..."
9477,246803,2023-02-01,Chorti,3,LAC,0,0.00,91317,0.37,101189,...,4936.0,0.02,0.0,0.0,0.22,0.22,0.63,0.61,0.63,"MULTIPOLYGON (((-89.35002 14.90313, -89.34886 ..."


In [20]:
# save the file
areas_gdf.to_file(r'C:\Users\WeilunShi\OneDrive - CGIAR\Desktop\Food Crisis and Insecurity\1.Initilization\intermediate\IPC_new.geojson', driver='GeoJSON')

In [21]:
from tqdm import tqdm
import fiona
import os

def convert_geojson_to_unique_polygons(input_geojson, output_shapefile):
    # Read the GeoJSON file and convert it to a GeoDataFrame
    geodata = gpd.read_file(input_geojson)

    # Filter by polygon geometry type
    filtered_geodata = geodata[geodata.geometry.type == 'Polygon']

    # Remove any other features except geometry
    filtered_geodata = filtered_geodata[['geometry']]

    # Drop duplicate geometries to get unique polygons
    filtered_geodata = filtered_geodata.drop_duplicates(subset=['geometry']).reset_index(drop=True)

    # Save the unique polygons as a shapefile
    if not filtered_geodata.empty:
        # Get the total number of features for the progress bar
        total_features = len(filtered_geodata)

        # Initialize the progress bar
        progress_bar = tqdm(total=total_features, desc='Converting unique polygons to Shapefile', unit='feature')

        # Prepare the output shapefile schema and CRS
        schema = {
            'geometry': 'Polygon',
            'properties': {}
        }
        crs = filtered_geodata.crs

        # Write the output shapefile in chunks
        with fiona.open(output_shapefile, 'w', driver='ESRI Shapefile', schema=schema, crs=crs) as dst:
            for feature in filtered_geodata.iterfeatures():
                dst.write(feature)
                progress_bar.update(1)

        # Close the progress bar
        progress_bar.close()
    else:
        print("No polygon features found in the input GeoJSON.")

os.chdir(r'C:\Users\WeilunShi\OneDrive - CGIAR\Desktop\Food Crisis and Insecurity\1.Initilization\intermediate')
input_geojson = 'IPC_new.geojson'
output_shapefile = 'unique_polygons_shapefile_new.shp'
convert_geojson_to_unique_polygons(input_geojson, output_shapefile)

Converting unique polygons to Shapefile: 100%|██████████| 2621/2621 [00:01<00:00, 1706.67feature/s]
